# Transcript Data Analytics

In [1]:
import os
import json

In [2]:
ROOT_DIR = os.path.dirname(os.path.dirname(os.getcwd()))
DATA_DIR = os.path.join(ROOT_DIR, 'data', 'podcasts-transcript', 'spotify-podcasts-2020')
TRANSCRIPT_ROOT_DIR = os.path.join(DATA_DIR, 'podcasts-transcripts')

In [3]:
# with open(os.path.join(DATA_DIR, 'file_list.txt'), 'r') as f:
#     first_line = next(f)

# print(first_line)
first_line = './podcasts-transcripts/0/E/show_0E2L8zPYhApYkmWWFef7aK/1RuLO0g4ps1p6kT3NB194o.json'


Pick a sample transcript JSON file

In [4]:
SAMPLE_PATH = os.path.join(DATA_DIR, first_line[2:].replace("\n", ''))

with open(SAMPLE_PATH, 'r') as f:
    data = json.load(f)

In [5]:
alternatives = data['results'][0]['alternatives']
alternatives[0].keys()

dict_keys(['transcript', 'confidence', 'words'])

## Structure of the transcript JSON file

* The file is a dict contianing a single key `results`, which is a list of items
* Each item in the list is a dictionary that has only one key `alternatives`
* This `alternatives` key refer to a list of **alternative** objects
* Each **alternative** is a dict whose keys include `transcript`, `confidence`, `words`
* `transcript`: an piece of texts corrresponding to a small portion of the transcript of each episode
* `confidence`: double
* `words`: a list of word token objects, each object has attributes `startTime`, `endTime` and `word`

Structure of the transcript JSON

In [6]:
results = data['results']
len(results)

17

### 1. A `result` item

Each `result` item contains a single key called `alternatives` which is a list

In [11]:
result = results[0]
result.keys()

dict_keys(['alternatives'])

### 2. An `alternative` item

Each `alternative` item is dictionary containing three keys
* `transcript`: an excerpt of the transcript
* `confidence`: confidence of the transcript (I guess it indicates how accurate the generated texts are)
* `words`: a list of JSON objects

In [13]:
alternatives = result['alternatives']
len(alternatives)

1

In [15]:
alternative = alternatives[0]
alternative.keys()

dict_keys(['transcript', 'confidence', 'words'])

In [16]:
alternative['transcript']

"Hello and welcome to the first episode of The Fan into flame podcast the official podcast of Greenwood Mennonite School athletics. My name is Tyler warfel. I teach high school math here at GMS and Coach the High School boys basketball team. I'm a big podcast listener. I listen to a lot of different types of podcasts mostly Sports podcast, but also some teaching podcast some news and current events podcasts. I like True Crime podcasts."

In [17]:
alternative['confidence']

0.853115439414978

In [19]:
len(alternative['words'])

76

### 3. A `word` item

Each word item is dict containing 3 required keys `startTime`, `endTime` and `word`. In some alternatives, a word can also contain the `speakerTag`

In [20]:
alternative['words'][:10]

[{'startTime': '1.400s', 'endTime': '1.700s', 'word': 'Hello'},
 {'startTime': '1.700s', 'endTime': '1.800s', 'word': 'and'},
 {'startTime': '1.800s', 'endTime': '2.200s', 'word': 'welcome'},
 {'startTime': '2.200s', 'endTime': '2.400s', 'word': 'to'},
 {'startTime': '2.400s', 'endTime': '2.400s', 'word': 'the'},
 {'startTime': '2.400s', 'endTime': '2.800s', 'word': 'first'},
 {'startTime': '2.800s', 'endTime': '3.500s', 'word': 'episode'},
 {'startTime': '3.500s', 'endTime': '3.600s', 'word': 'of'},
 {'startTime': '3.600s', 'endTime': '3.800s', 'word': 'The'},
 {'startTime': '3.800s', 'endTime': '4.300s', 'word': 'Fan'}]

## Anomaly detection

We suspect that not all **alternatives** have the above structure, let's see if there is something weird

In [21]:
multi_al_results = [res for res in results if len(res['alternatives']) != 1]
multi_al_results

[]

In [24]:
res_alternatives = [res['alternatives'][0] for res in results]
len(res_alternatives)

17

In [26]:
for alt in res_alternatives:
    if 'transcript' not in alt.keys():
        print("Alternative doesn't have `transcript` key")
        print("No. of words: ", len(alt['words']))

Alternative doesn't have `transcript` key
No. of words:  1283


## Conclusion

After the above exploratory analysis, we conclude that:
* Each JSON transcript file hold a single dictionary that contains a single key `results`
* The `results` is a list of dictionary that contains a single key `alternatives` which in tern is a list of object
* Each `alternatives` contains a list of dictionary, dubbed `alternative`
* There are **two** types of `alternative`:
  - Type 1: a dictionary containing **three** keys: `transcript`, `confidence`, and `words`
  - Type 2: a dictionary containing **only one** key `words` that hold all the word tokens in the transcript of a podcast